### reading the video, extracting, first frame 
here the video is loaded, and the first frame extracted, the frame used later for the homography transformation


In [22]:
video.release()

In [16]:
import cv2 
import numpy as np

video = cv2.VideoCapture(r"../videos/resized_gameplay_3.mp4")
original_fps = video.get(cv2.CAP_PROP_FPS)
print(f'the fps rate in the recorded video is {original_fps}')

# Check if the video file opened successfully
if not video.isOpened():
    print("Error opening the video file")
    exit()

# Display the first frame
while video.isOpened():
    ret, frame = video.read()
    if ret:
        cv2.imshow('Frame', frame)
        
        # Wait for user input
        key = cv2.waitKey(0)  # Wait indefinitely for a key press
        # if key == ord('f'):  # Close the window if 'f' is pressed
        break
    else:
        print("Unable to read the video frame.")
        break
cv2.destroyAllWindows()
# cv2.waitKey(0)

the fps rate in the recorded video is 30.013


### Here the canny detector, and hugh lines tried to manually extract valuable features from the frame, but it doesnt seem promising the keypoints seem much better

In [17]:



frame_grayscale =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
frame_gray = frame_grayscale.copy()
#parameters for 256x256
#corners = cv2.goodFeaturesToTrack(frame_grayscale, maxCorners=50, qualityLevel=0.1, minDistance=100)
corners = cv2.goodFeaturesToTrack(frame_grayscale, maxCorners=4, qualityLevel=0.1, minDistance=200)
corners = np.intp(corners)

# Draw corners
for i in corners:
    x, y = i.ravel()
    cv2.circle(frame_grayscale, (x, y), 5, 255, -1)

cv2.imshow('Corners', frame_grayscale)
cv2.waitKey(0)

edges = cv2.Canny(frame_gray, 50, 150)
#for 256 x 256
#lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)
lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

# Draw detected lines
for rho, theta in lines[:, 0]:
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = a * rho, b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * a)
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * a)
    cv2.line(frame_gray, (x1, y1), (x2, y2), (255, 0, 0), 2)

cv2.imshow('Lines', frame_gray)
cv2.waitKey(0)

# video.release()

reference_pick = cv2.imread("../images/board_image.jpg")
reference_pick= cv2.resize(reference_pick, (876,876), interpolation = cv2.INTER_AREA)
reference_pick = cv2.cvtColor(reference_pick,cv2.COLOR_BGR2GRAY)
cv2.destroyAllWindows()











### key points finding  and matching them

here , I try to find the keypoints on both pictures and match them, to be able to now the transformation between the reference image and the video, thus mapp coordinates or compute the homographic transform

In [18]:
def show_matches(bf, kp1, desc1, kp2, desc2, num_matches=30):
    matches = bf.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    pts_ref = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts_frame = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, _ = cv2.findHomography(pts_frame, pts_ref, cv2.RANSAC, 5.0)

    # Warp video frame to align with reference image
    aligned_frame = cv2.warpPerspective(frame, H, (reference_pick.shape[1], reference_pick.shape[0]))
    
    matches = cv2.drawMatches(
        reference_pick,
        kp1,
        frame,
        kp2,
        matches[:num_matches],
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    )
    cv2.imshow("matches",matches)
    cv2.waitKey(0)
    cv2.imshow('Aligned Frame', aligned_frame)
    cv2.waitKey(0)
    

In [19]:


orb = cv2.ORB_create()
sift = cv2.SIFT_create()  # kp + desc
fast = cv2.FastFeatureDetector_create()  # only kp
star = cv2.xfeatures2d.StarDetector_create()  # only kp
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()  # only desc


# Detect and compute keypoints and descriptors
reference_pick = cv2.imread("../images/board_image.jpg")
reference_pick = cv2.resize(reference_pick, (512,512), interpolation = cv2.INTER_AREA)


orb_kp1, orb_des1 = orb.detectAndCompute(reference_pick, None)
orb_kp2, orb_des2 = orb.detectAndCompute(frame, None)

sift_kp1, sift_des1 = sift.detectAndCompute(reference_pick, None)
sift_kp2, sift_des2 = sift.detectAndCompute(frame, None)

fast_kp1 = fast.detect(reference_pick, None)
fast_kp2 = fast.detect(frame, None)

star_kp1 = star.detect(reference_pick, None)
star_kp2 = star.detect(frame, None)

_, fast_brief_des1 = brief.compute(reference_pick, fast_kp1)
_, fast_brief_des2 = brief.compute(frame, fast_kp2)

_, star_brief_des1 = brief.compute(reference_pick, star_kp1)
_, star_brief_des2 = brief.compute(frame, star_kp2)


# Match features using BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
bf_l2 = cv2.BFMatcher(cv2.NORM_L2,crossCheck=True)


show_matches(bf_l2,sift_kp1,sift_des1,sift_kp2,sift_des2,50)
# cv2.waitKey(0)
show_matches(bf,orb_kp1,orb_des1,orb_kp2,orb_des2,50)
# cv2.waitKey(0)
show_matches(bf_l2,orb_kp1,orb_des1,orb_kp2,orb_des2,50)
# cv2.waitKey(0)
show_matches(bf,fast_kp1,fast_brief_des1,fast_kp2,fast_brief_des2,50)
# cv2.waitKey(0)
show_matches(bf_l2,fast_kp1,fast_brief_des1,fast_kp2,fast_brief_des2,50)
# cv2.waitKey(0)
show_matches(bf,star_kp1,star_brief_des1,star_kp2,star_brief_des2,50)
# cv2.waitKey(0)
show_matches(bf_l2,star_kp1,star_brief_des1,star_kp2,star_brief_des2,50)
# cv2.waitKey(0)




# 1,3,6 worth looking at

cv2.destroyAllWindows()


## contour detection
The idea is to , before finding the keypoints, find the biggest contour on the frame from the video, in order to search for the keypoints only within this contour - which corresponds to the board isolated from the -environment - i.e the table 

In [20]:
reference_pick = cv2.imread("../images/board_image.jpg")
reference_pick = cv2.resize(reference_pick, (512,512), interpolation = cv2.INTER_AREA)
reference_pick = cv2.cvtColor(reference_pick,cv2.COLOR_BGR2GRAY)
cv2.imshow("original frame",frame)
cv2.waitKey(0)
frame_gray =cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.imshow("before_clahe",frame_gray)
cv2.waitKey(0)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
frame_gray = clahe.apply(frame_gray)
cv2.imshow("after_clahe",frame_gray)
cv2.waitKey(0)
frame_gray = cv2.GaussianBlur(frame_gray,(3,3),0)
cv2.imshow("after_gaussian",frame_gray)
cv2.waitKey(0)
_,thresh = cv2.threshold(frame_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
cv2.drawContours(frame_gray, [largest_contour], -1, (0, 255, 0), 3)


cv2.imshow('Largest Contour', frame_gray)
cv2.waitKey(0)


empty_image = np.zeros_like(frame_gray)
cv2.drawContours(empty_image, [largest_contour], -1, (255, 255, 255), 3)
cv2.imshow('Largest Contour', empty_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


### keypoints detection using the extracted mask, restricting the area of the keypoints search in the frame

having the mask we can search for the keypoints only within it , which  decreases the chance of wrongfull matching

In [21]:
mask = np.zeros_like(frame_gray, dtype=np.uint8)

# Fill the largest contour on the mask
cv2.drawContours(mask, [largest_contour], -1, 255, -1)  # Fill the contour with white
cv2.imshow("mask",mask)
cv2.waitKey(0)
orb = cv2.ORB_create()
sift = cv2.SIFT_create()  # kp + desc
fast = cv2.FastFeatureDetector_create()  # only kp
star = cv2.xfeatures2d.StarDetector_create()  # only kp
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()  # only desc


# Detect and compute keypoints and descriptors
reference_pick = cv2.imread("../images/board_image.jpg")
reference_pick = cv2.resize(reference_pick, (512,512), interpolation = cv2.INTER_AREA)



orb_kp1, orb_des1 = orb.detectAndCompute(reference_pick, None)
orb_kp2, orb_des2 = orb.detectAndCompute(frame, mask)
sift_kp1, sift_des1 = sift.detectAndCompute(reference_pick, None)
sift_kp2, sift_des2 = sift.detectAndCompute(frame, mask)
fast_kp1 = fast.detect(reference_pick, None)
fast_kp2 = fast.detect(frame, mask)
check = fast.detect(frame, None)
star_kp1 = star.detect(reference_pick, None)
star_kp2 = star.detect(frame, mask)

# Compute descriptors for FAST and STAR keypoints
_, fast_brief_des2 = brief.compute(frame, fast_kp2)
_, star_brief_des2 = brief.compute(frame, star_kp2)

# Match features using BFMatcher (as before)
show_matches(bf_l2, sift_kp1, sift_des1, sift_kp2, sift_des2, 50)
show_matches(bf, orb_kp1, orb_des1, orb_kp2, orb_des2, 50)
show_matches(bf_l2, orb_kp1, orb_des1, orb_kp2, orb_des2, 50)
show_matches(bf, fast_kp1, fast_brief_des1, fast_kp2, fast_brief_des2, 50)
show_matches(bf_l2, fast_kp1, fast_brief_des1, fast_kp2, fast_brief_des2, 50)
show_matches(bf, star_kp1, star_brief_des1, star_kp2, star_brief_des2, 50)
show_matches(bf_l2, star_kp1, star_brief_des1, star_kp2, star_brief_des2, 50)
cv2.destroyAllWindows()

### filtering the image based on the rgb/hsv/hls features, which may be usefull to extract later the board from the frame in the vido, and only then do the keypoint analysis and  transform /match
The colorFinder class enables to manually find the range of values for the features that interest us and filter those values in image, thus potentially enabling to distinguish the board from the background 



In [14]:


class ColorFinder:
    def __init__(self, trackBar=False, colorSpace='HSV'):
        """
        :param trackBar: Whether to use OpenCV trackbars to dynamically adjust HSV/HLS/GBR values. Default is False.
        :param colorSpace: The color space to use, either 'HSV', 'HLS', or 'GBR'. Default is 'HSV'.
        """
        self.trackBar = trackBar
        self.colorSpace = colorSpace.upper()  # Convert to uppercase for consistency
        if self.trackBar:
            self.initTrackbars()

    def empty(self, a):
        pass

    def initTrackbars(self):
        """Initialize the OpenCV trackbars for dynamic HSV/HLS/GBR value adjustment."""
        cv2.namedWindow("TrackBars")
        cv2.resizeWindow("TrackBars", 640, 240)
        

        if self.colorSpace == 'HLS':
            cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, self.empty)
            cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, self.empty)
            cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Light Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Light Max", "TrackBars", 255, 255, self.empty)
        elif self.colorSpace == 'GBR':
            # Add trackbars for Green, Blue, and Red channels
            cv2.createTrackbar("Green Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Green Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Blue Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Blue Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Red Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Red Max", "TrackBars", 255, 255, self.empty)
        else:
            cv2.createTrackbar("Hue Min", "TrackBars", 0, 179, self.empty)
            cv2.createTrackbar("Hue Max", "TrackBars", 179, 179, self.empty)
            cv2.createTrackbar("Sat Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Sat Max", "TrackBars", 255, 255, self.empty)
            cv2.createTrackbar("Value Min", "TrackBars", 0, 255, self.empty)
            cv2.createTrackbar("Value Max", "TrackBars", 255, 255, self.empty)

    def getTrackbarValues(self):
        """Get the current color values set by the trackbars."""
       

        if self.colorSpace == 'HLS':
            hmin = cv2.getTrackbarPos("Hue Min", "TrackBars")
            smin = cv2.getTrackbarPos("Sat Min", "TrackBars")
            hmax = cv2.getTrackbarPos("Hue Max", "TrackBars")
            smax = cv2.getTrackbarPos("Sat Max", "TrackBars")
            lmin = cv2.getTrackbarPos("Light Min", "TrackBars")
            lmax = cv2.getTrackbarPos("Light Max", "TrackBars")
            colorVals = {"hmin": hmin, "smin": smin, "lmin": lmin,
                         "hmax": hmax, "smax": smax, "lmax": lmax}
        elif self.colorSpace == 'GBR':
            gmin = cv2.getTrackbarPos("Green Min", "TrackBars")
            gmax = cv2.getTrackbarPos("Green Max", "TrackBars")
            bmin = cv2.getTrackbarPos("Blue Min", "TrackBars")
            bmax = cv2.getTrackbarPos("Blue Max", "TrackBars")
            rmin = cv2.getTrackbarPos("Red Min", "TrackBars")
            rmax = cv2.getTrackbarPos("Red Max", "TrackBars")
            colorVals = {"gmin": gmin, "bmin": bmin, "rmin": rmin,
                         "gmax": gmax, "bmax": bmax, "rmax": rmax}
        else:
            hmin = cv2.getTrackbarPos("Hue Min", "TrackBars")
            smin = cv2.getTrackbarPos("Sat Min", "TrackBars")
            hmax = cv2.getTrackbarPos("Hue Max", "TrackBars")
            smax = cv2.getTrackbarPos("Sat Max", "TrackBars")
            vmin = cv2.getTrackbarPos("Value Min", "TrackBars")
            vmax = cv2.getTrackbarPos("Value Max", "TrackBars")
            colorVals = {"hmin": hmin, "smin": smin, "vmin": vmin,
                         "hmax": hmax, "smax": smax, "vmax": vmax}

        return colorVals

    def update(self, img, myColor=None):
        """Find a specified color in the given image."""
        imgColor = []
        mask = []

        if self.trackBar:
            myColor = self.getTrackbarValues()

        if myColor is not None:
            # If using GBR, directly use the image as it is in GBR format (OpenCV loads images as BGR)
            if self.colorSpace == 'HLS':
                imgColorSpace = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
            elif self.colorSpace == 'GBR':
                imgColorSpace = img  # No conversion needed, as we are working with BGR (GBR in OpenCV terms)
            else:  # HSV
                imgColorSpace = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            # Apply the color filtering for GBR
            if self.colorSpace == 'GBR':
                lower = np.array([myColor['gmin'], myColor['bmin'], myColor['rmin']])
                upper = np.array([myColor['gmax'], myColor['bmax'], myColor['rmax']])
            elif self.colorSpace == 'HLS':
                lower = np.array([myColor['hmin'], myColor['lmin'], myColor['smin']])
                upper = np.array([myColor['hmax'], myColor['lmax'], myColor['smax']])
            else:  # HSV
                lower = np.array([myColor['hmin'], myColor['smin'], myColor['vmin']])
                upper = np.array([myColor['hmax'], myColor['smax'], myColor['vmax']])

            # Create a mask and apply it to the image
            mask = cv2.inRange(imgColorSpace, lower, upper)
            imgColor = cv2.bitwise_and(img, img, mask=mask)

        return imgColor, mask


In [15]:
def find_values(image,colorspace='gbr'):
    if colorspace == 'hsv':
         image = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    elif colorspace == 'hls':
         image = cv2.cvtColor(image,cv2.COLOR_BGR2HLS)
    image = cv2.GaussianBlur(image,(5,5),0)
    if colorspace == 'hls':
        colorFinder = ColorFinder(trackBar=True,colorSpace='HLS')
    elif colorspace == 'gbr':
         colorFinder = ColorFinder(trackBar=True,colorSpace='GBR')
    else:
        colorFinder = ColorFinder(trackBar=True)
    while True:
            # Get the current color values from trackbars
            hsvVals = colorFinder.getTrackbarValues()

            # Update the image with the new HSV values
            imgProcessed, mask = colorFinder.update(image, hsvVals)

            # Display the original image and the processed image
            cv2.imshow("Original Image", image)
            cv2.imshow("Processed Image", imgProcessed)

            # Exit if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release resources
    cv2.destroyAllWindows()
    return hsvVals
    
def show_filtered_values(image,vals,colorspace='gbr'):
    if colorspace == 'hsv':
        image= cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        colorFinder = ColorFinder(trackBar=False,colorSpace='HSV')
    elif colorspace == 'hls':
        image= cv2.cvtColor(image,cv2.COLOR_BGR2HLS)
        colorFinder = ColorFinder(trackBar=False,colorSpace='HLS')
    else:
        colorFinder = ColorFinder(trackBar=False,colorSpace='GBR')   
    # colorFinder = ColorFinder(trackBar=False)
    imgProcessed, mask = colorFinder.update(image, vals)
    cv2.imshow("filtered image",imgProcessed)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return 

'''one can play around with the parameters, but so far only hsv seemed usefull somehow?'''

'''usage, in the hsv space'''
found_vals = find_values(frame,colorspace="hsv")
print(found_vals)
show_filtered_values(frame,found_vals,colorspace='hsv')
'''usage, in the gbr space, default'''
# so maybe we can manually find the color of the table, and  filter out this color on the image , interesting idea 
found_vals = find_values(frame)
print(found_vals)
show_filtered_values(frame,found_vals)
'''usage ,in the hls space'''
found_vals = find_values(frame,colorspace='hls')
print(found_vals)
show_filtered_values(frame,found_vals,colorspace='hls')

{'hmin': 17, 'smin': 0, 'vmin': 0, 'hmax': 179, 'smax': 255, 'vmax': 255}
{'gmin': 102, 'bmin': 0, 'rmin': 0, 'gmax': 255, 'bmax': 255, 'rmax': 255}
{'hmin': 0, 'smin': 0, 'lmin': 33, 'hmax': 179, 'smax': 255, 'lmax': 255}
